# Overview of Marlowe CLI

This lecture provides an overview of the low and high levels at which one can create and interact with Marlowe contracts on the blockchain using the Marlowe CLI tool, and how that relates to using Marlowe Run or the Marlowe Playground's simulator. It introduces the `marlowe-cli` tool and includes instructions for installing and using it. Subsequent lectures focus on particular levels of interaction with Marlowe contract using `marlowe-cli`.

## Purpose

The `marlowe-cli` tool enables developers to submit transactions and interact from the command line with Marlowe contracts on the Cardano blockchain, just as the `cardano-cli` tool has enabled them to do so for plain transactions, simple scripts, and Plutus scripts.

***Use Cases***

1.  Facilitation of internal development and testing of Marlowe contracts.
    1.  Measuring transaction size.
    2.  Submitting transactions.
    3.  Testing wallet integrations.
    4.  Debugging validators.
2.  Early access to Marlowe capabilities on testnet and mainnet.
3.  Integration with external developer’s workflows and toolsets for Marlowe contracts, similar to how the Cardano development community has heavily integrated `cardano-cli` into various services (libraries, faucets, token minting, marketplaces, etc.).
4.  Incorporation into training for use of Marlowe.

## Three Levels of Interaction with Marlowe on the Blockchain

| Low Level | High Level | Plutus Application Backend (PAB) |
|-----------|------------|----------------------------------|
| • Supports developer workflows for debugging and fine-grained control of each atomic operation involved in running Marlowe contracts. | • Supports a straightforward workflow for users that just want to run contracts from the command line. | • Supports direct interaction with the PAB contracts and endpoints for running Marlowe from the command line. |
| • Controls modification of Marlowe state and construction of Marlowe input. | • Hides details of input and state of Marlowe contracts. | • Hides details of UTxO management by interacting with a Cardano Wallet or Daedalus. |
| • Controls construction and submission of validators, datums, and redeemers. | • Hides and automates many aspects of Plutus and interaction with the Cardano node. | • Mimics the workflow for using Marlowe Run. |
| • Focus on the mechanics of Marlowe on Plutus and Cardano. | • Focuses on the Marlowe contract. | • Focuses on user interaction. |

## Review of eUTxO Model

![Boxes are UTxOs; ovals are transactions.](diagrams/eutxo.png)

## Marlowe Roles: Currency, Tokens, and Payouts

*   Each Marlowe contract has a currency symbol for its roles and a payout validator script for receiving and disbursing payments to participants.
*   Participants typically interact with Marlowe contracts by including a role token in transactions that the participant authorizes.
    *   Role tokens are a concatenation of the currency symbol and the token name.
*   Participants withdraw funds from a special payout script using their role token.
    *   The payout Plutus script is unique to each role currency symbol.
    *   The datum at the payout address is simply the token name for the participant that is allowed to spend that UTxO.
    *   The redeemer for spending from the payout script is empty.
*   Roles provide a flexible and secure way for participants to interact with Marlowe.
*   Role tokens can be transferred from one wallet to another.

## Marlowe on the Blockchain

| Component | Plutus Validator Script | Datum | Redeemer |
|-----------|-------------------------|-------|----------|
| Minting role tokens | One-shot monetary policty | n/a | n/a |
| Marlowe application | Generic for all Marlowe contracts, but specialized to the roles for the contract | Unexecuted parts of the contract; balances for accounts; records of past inputs; values of parameters | Input to the next step of the contract,such as deposits, choices, or notifications |
| Marlowe payouts | Specific to particular role currency | Role that is to be paid. | n/a |

## Life Cycle of a Marlowe Contract

![Transitions in a Marlowe contract.](diagrams/lifecycle.png)

### Using Marlowe CLI to Study Transitions of a Marlowe Contract

![`marlowe-cli` commands transitioning a Marlowe contract without submitting transactions.](diagrams/high-level-abstract.png)

### Using Marlowe CLI to Execute Transactions for a Marlowe Contract

![`marlowe-cli` commands for creating and submitting transactions for a Marlowe contract.](diagrams/high-level-concrete.png)

### Transaction that Creates a Marlowe Contract

Use `marlowe-cli run execute` to create a Marlowe contract.

![Boxes are UTxOs; ovals are transactions; ADA fees are approximate.](diagrams/create.png)

### Transaction that Applies Inputs to a Marlowe Contract

Use `marlowe-cli run execute` to apply inputs to a Marlowe contract.

![Boxes are UTxOs; ovals are transactions; ADA fees are approximate.](diagrams/apply-inputs.png)

### Transaction that Redeems a Marlowe Payout

Use `marlowe-cli run withdraw` to redeem funds of a Marlowe payout.

![Boxes are UTxOs; ovals are transactions; ADA fees are approximate.](diagrams/redeem.png)

## Escrow Example

*   A buyer wishes to purchase an item.
*   They deposit funds.
*   After they deposit funds they may report a problem with the purchase.
*   If they don’t report a problem, then the funds are released to the seller.
*   If they do report a problem, the seller may agree that there is a problem (in which case the buyer receives a refund) or they may dispute that there is a problem.
*   If the seller disputes the problem, then a mediator decides who receives the funds.
*   The contract has logic to handle situations where a party fails to act in a timely manner.

### Flow Chart for Escrow Example

![Execution paths through the escrow example contract.](diagrams/escrow-flow.png)

### Escrow Example in Marlowe Format

```
When
  [
    Case (Deposit (Role "Francis Beaumont (the seller)") (Role "Thomas Middleton (the buyer)") ada 256)
      ( When
        [
          Case (Choice (ChoiceId "Everything is alright" (Role "Thomas Middleton (the buyer)")) [Bound 0 0])
            Close
        , Case (Choice (ChoiceId "Report problem" (Role "Thomas Middleton (the buyer)")) [Bound 1 1])
            ( Pay (Role "Francis Beaumont (the seller)") (Account (Role "Thomas Middleton (the buyer)")) ada 256 )
            ( When
              [
                Case (Choice (ChoiceId "Confirm problem" (Role "Francis Beaumont (the seller)")) [Bound 1 1])
                  Close
              , Case (Choice (ChoiceId "Dispute problem" (Role "Francis Beaumont (the seller)")) [Bound 0 0])
                ( When
                  [
                    Case (Choice (ChoiceId "Dismiss claim" (Role "Christopher Marlowe (the mediator)")) [Bound 0 0])
                      ( Pay (Role "Thomas Middleton (the buyer)") (Account (Role "Francis Beaumont (the seller)")) ada 256 )
                      Close
                  , Case (Choice (ChoiceId "Confirm claim" (Role "Christopher Marlowe (the mediator)")) [Bound 1 1])
                      Close
                  ]
                  (SlotParam "Mediation deadline")
                  Close
                )
              ]
              (SlotParam "Dispute deadline")
            ) Close
        ]
        (SlotParam "Complaint deadline")
        Close
      )
  ]
  (SlotParam "Payment deadline")
  Close
```

### Escrow Example in Blockly

![Blockly representation of escrow example contract, split into three columns.](diagrams/escrow-blockly.png)

### UTxOs for One Path through Escrow example

![Boxes are UTxOs; ovals are transactions; ADA values are rounded to one decimal place.](diagrams/escrow-utxos.png)

## Installing and Running Marlowe CLI

The Marlowe CLI tool is written in the Haskell language. Here are three methods for installing `marlowe-cli`, listed in order of increasing difficulty.
*   *Via Daedalus:* A copy of `marlowe-cli` is provided with the Daedalus wallet for Marlowe Pioneers.
*   *Via Nix:* This requires running either NixOs or the Nix package manager. See <<https://nixos.org/>>.
*   *Via Cabal:* This requires installing the Haskell tookchain. See <<https://www.haskell.org/downloads/>> for installing GHC 8.10.7 and Cabal 3.6.

The last two installation methods require cloning the git repository for Marlowe:

In [1]:
git clone git@github.com:input-output-hk/marlowe-cardano.git

fatal: destination path 'marlowe-cardano' already exists and is not an empty directory.


: 128

👉 *Troubleshooting*
*   See instructions for installing `marlowe-cli`: ***TO BE WRITTEN***.
*   The read-me document for the Marlowe project contains extensive details on building Marlowe components such as `marlowe-cli`: <<https://github.com/input-output-hk/marlowe-cardano/blob/main/README.adoc>>.

### Installation via Daedalus

***TO BE WRITTEN***

### Installation via Nix

See <<https://github.com/input-output-hk/marlowe-cardano/blob/main/README.adoc#how-to-set-up-the-iohk-binary-caches>> on how to set up binary caches for the Nix build. This will greatly speed the build process.

Once the caches have been set up, simply build the `marlowe-cli` executable.

In [1]:
nix-build marlowe-cardano/default.nix -A marlowe-cli -o build

/nix/store/6lcpxll8l6wcm298kagjkif3xix9crax-marlowe-cli-exe-marlowe-cli-0.0.0.1


Add `marlowe-cli` to the shell's path.

In [2]:
export PATH=$PWD/build/bin:$PATH

Check that `marlowe-cli` has been installed and is available on the shell's path.

In [4]:
marlowe-cli --version

marlowe-cli 0.0.3.1 @ 


### Installation via Cabal

Check that version 8.10.7 of GHC and version 3.6 of Cabal have been installed.

In [2]:
ghc --version
cabal --version

The Glorious Glasgow Haskell Compilation System, version 8.10.7
cabal-install version 3.6.0.0
compiled using version 3.6.1.0 of the Cabal library 


Build the Marlowe CLI tool, which will take a long time.

In [ ]:
pushd marlowe-cardano > /dev/null
cabal install exe:marlowe-cli
popd

Add `marlowe-cli` to the shell's path.

If the Cabal installation directory is on the shell's path, then the `marlowe-cli` command should work.

In [5]:
marlowe-cli --version

marlowe-cli 0.0.3.1 @ 


## Getting Help for Marlowe CLI

In [1]:
marlowe-cli --help

marlowe-cli : a command-line tool for Marlowe contracts

Usage: marlowe-cli [--version] (COMMAND | COMMAND)

  Utilities for Marlowe.

Available options:
  -h,--help                Show this help text
  --version                Show version.

High-level commands:
  run                      Run a contract.
  pab                      Run a contract via the PAB.
  template                 Create a contract from a template.
  test                     Test contracts.

Low-level commands:
  contract                 Export contract address, validator, datum, or
                           redeemer.
  input                    Create inputs to a contract.
  role                     Export role address, validator, datum, or redeemer.
  query                    Blockchain queries for Marlowe.
  transaction              Create and submit transactions.
  util                     Miscellaneous utilities.


## Next Lectures

*   Using `marlowe-cli` to run the escrow example.
    *   Without submitting transactions.
    *   With submitting transactions.
*   Using `marlowe-cli` to run contracts using the Plutus Application Backend (PAB).
*   Reference for `marlowe-cli` commands.